<img src="../../../images/banners/data_processing.png" width="600"/>

# <img src="../../../../images/logos/python.png" width="23"/> An Array of Sequences 


## <img src="../../../../images/logos/toc.png" width="20"/> Table of Contents 
* [When a List Is Not the Answer](#when_a_list_is_not_the_answer)
* [Overview of Built-In Sequences](#overview_of_built-in_sequences)
* [Arrays](#arrays)
    * [Memory Views](#memory_views)

---

Before creating Python, Guido was a contributor to the ABC language—a 10-year research project to design a programming environment for beginners. ABC introduced many ideas we now consider “Pythonic”: generic operations on different types of sequences, built-in tuple and mapping types, structure by indentation, strong typing without variable declarations, and more. It’s no accident that Python is so user-friendly.

Python inherited from ABC the uniform handling of sequences. Strings, lists, byte sequences, arrays, XML elements, and database results share a rich set of common operations, including iteration, slicing, sorting, and concatenation.

Understanding the variety of sequences available in Python saves us from reinventing the wheel, and their common interface inspires us to create APIs that properly support and leverage existing and future sequence types.

Most of the discussion in this chapter applies to `list` and `array`. Other general sequences such as `tuple` and `queue` are not for discussion here.

<a class="anchor" id="when_a_list_is_not_the_answer"></a>

## When a List Is Not the Answer

The `list` type is flexible and easy to use, but depending on specific requirements, there are better options. For example, an `array` saves a lot of memory when you need to handle millions of floating-point values (arrays will be discussed later in details). On the other hand, if you are constantly adding and removing items from opposite ends of a list, it’s good to know that a `deque` (double-ended queue) is a more efficient FIFO14 data structure.

<a class="anchor" id="overview_of_built-in_sequences"></a>

## Overview of Built-In Sequences

The standard library offers a rich selection of sequence types implemented in C:

- Container sequences
> Can hold items of different types, including nested containers. Some examples: `list`, `tuple`, and `collections.deque`.

- Flat sequences
> Hold items of one simple type. Some examples: `str`, `bytes`, and `array.array`.

A **container sequence** holds references to the objects it contains, which may be of any type, while a **flat sequence** stores the value of its contents in its own memory space, not as distinct Python objects. See Figure 2-1.

<img src="../images/an-array-of-sequences/array_list.png" width="600"/>

Thus, `flat` sequences are more compact, but they are limited to holding primitive machine values like bytes, integers, and floats.

Every Python object in memory has a header with metadata. The simplest Python object, a float, has a value field and two metadata fields:

- `ob_refcnt`: the object’s reference count
- `ob_type`: a pointer to the object’s type
- `ob_fval`: a C `double` holding the value of the `float`

On a 64-bit Python build, each of those fields takes 8 bytes. That’s why an array of floats is much more compact than a tuple of floats: the array is a single object holding the raw values of the floats, while the tuple consists of several objects—the tuple itself and each float object contained in it.

For the remainder of this chapter, we discuss mutable sequence types that can replace lists in many cases, starting with arrays.

<a class="anchor" id="arrays"></a>

## Arrays

If a list only contains numbers, an `array.array` is a more efficient replacement. Arrays support all mutable sequence operations (including `.pop`, `.insert`, and `.extend`), as well as additional methods for fast loading and saving, such as `.frombytes` and `.tofile`.

Arrays are sequence types and behave very much like lists, except that the type of objects stored in them is constrained. The type is specified at object creation time by using a type code, which is a single character. The following type codes are defined:

| Type code| C Type| Python Type| Minimum size in bytes|
|:-- |:-- |:-- |:-- |
| `'b'` | signed char | int | 1 |
| `'B'`| unsigned char| int| 1 |
| `'u'` | wchar_t | Unicode character | 2 |
| `'h'` | signed short | int | 2 |
| `'H'` | unsigned short | int | 2 |
| `'i'` | signed int | int | 2 |
| `'I'` | unsigned int | int | 2 |
| `'l'` | signed long | int | 4 |
| `'L'` | unsigned long | int | 4 |
| `'q'` | signed long long | int | 8 |
| `'Q'` | unsigned long long | int | 8 |
| `'f'` | float | float | 4 |
| `'d'` | double | float | 8 |

A Python array is as lean as a C array. As shown in the image above, an `array` of `float` values does not hold full-fledged `float` instances, but only the packed bytes representing their machine values—similar to an array of `double` in the C language. When creating an `array`, you provide a typecode, a letter to determine the underlying C type used to store each item in the array. For example, `b` is the typecode for what C calls a `signed char`, an integer ranging from –128 to 127. If you create an `array('b')`, then each item will be stored in a single byte and interpreted as an integer. For large sequences of numbers, this saves a lot of memory. And Python will not let you put any number that does not match the type for the array.

In [38]:
from array import array
from random import random

In [4]:
floats = array('d', (random() for i in range(10**7)))

In [7]:
floats[-1]

0.09549897660590423

In [8]:
with open('floats.bin', 'wb') as f:
    floats.tofile(f)

In [9]:
floats_2 = array('d')

In [10]:
with open('floats.bin', 'rb') as f:
    floats_2.fromfile(f, 10**7)

In [12]:
# Verify that the contents of the arrays match.
floats_2[-1]

0.09549897660590423

In [13]:
floats_2 == floats

True

As you can see, `array.tofile` and `array.fromfile` are easy to use. If you try the example, you’ll notice they are also very fast. A quick experiment shows that it takes about 0.1 seconds for `array.fromfile` to load 10 million double-precision floats from a binary file created with `array.tofile`. That is nearly **60 times faster than reading the numbers from a text file**, which also involves parsing each line with the float built-in. Saving with `array.tofile` is about **seven times faster than writing one float per line in a text file**. In addition, the size of the binary file with 10 million doubles is **80,000,000 bytes (8 bytes per double, zero overhead), while the text file has 181,515,739 bytes for the same data**.


If you do a lot of work with arrays and don’t know about memoryview, you’re missing out. See the next topic.

<a class="anchor" id="memory_views"></a>

### Memory Views

A `memoryview` is essentially a generalized NumPy array structure in Python itself (without the math). It allows you to share memory between data-structures (things like PIL images, SQLlite data-bases, NumPy arrays, etc.) without first copying. This is very important for large data sets.

With it you can do things like memory-map to a very large file, slice a piece of that file and do calculations on that piece (easiest if you are using NumPy).

The built-in `memoryview` class is a shared-memory sequence type that lets you handle slices of arrays without copying bytes. It was inspired by the NumPy library (which we’ll discuss shortly in “NumPy”). Travis Oliphant, lead author of [NumPy](https://learning.oreilly.com/library/view/fluent-python-2nd/9781492056348/ch02.html#numpy_sec), answers the question, [“When should a memoryview be used?”](https://fpy.li/2-17) like this:

> A memoryview is essentially a generalized NumPy array structure in Python itself (without the math). It allows you to share memory between data-structures (things like PIL images, SQLite databases, NumPy arrays, etc.) without first copying. This is very important for large data sets.

Using notation similar to the `array` module, the `memoryview.cast` method lets you change the way multiple bytes are read or written as units without moving bits around. `memoryview.cast` returns yet another `memoryview` object, always sharing the same memory.

In [30]:
from array import array
octets = array('b', range(6))

In [31]:
m1 = memoryview(octets)

In [32]:
m1.tolist()

[0, 1, 2, 3, 4, 5]

In [33]:
m2 = m1.cast('b', [2, 3])
m3 = m1.cast('B', [3, 2])

In [34]:
m2.tolist()

[[0, 1, 2], [3, 4, 5]]

In [35]:
m2[1, 1] = 22

In [36]:
m3[1,1] = 33

In [37]:
octets

array('b', [0, 1, 2, 33, 22, 5])

Meanwhile, if you are doing advanced numeric processing in arrays, you should be using the NumPy libraries. We’ll cover numpy arrays later.